In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_dir = '/content/drive/MyDrive/Models_exp2.11'

In [3]:
import os
import multiprocessing
import numpy as np
import cv2
import time
import shutil

import torch

import keras
from tensorflow.keras.applications import EfficientNetB4

In [4]:
!nvidia-smi

Fri Apr 30 20:07:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
#https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
#efficient nets are a lot easier to work with in keras
from keras import layers

def build_model(num_classes, IMG_SIZE):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    eff_model = EfficientNetB4(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    eff_model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(eff_model.output)
    x = layers.BatchNormalization()(x)

    # top_dropout_rate = 0.2
    # x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=.9)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [7]:
views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top', 'top_1', 'top_2', 'top_3', 'top_4', 'bot_1', 'bot_2', 'bot_3', 'bot_4']
#np.savez_compressed('/content/drive/MyDrive/Cache/keras/train_val', x=x, y=y, x_val=val_x, y_val=val_y)
file = np.load('/content/drive/MyDrive/Cache/keras/train_val.npz')
x = file['x']
y = file['y']
x_val = file['x_val']
y_val = file['y_val']

In [8]:
#https://stackoverflow.com/questions/60567679/save-keras-model-weights-directly-to-bytes-memory
class Save_Best_Weights(keras.callbacks.Callback):   
  def __init__(self, min=True, metric='val_loss'):
      super(Save_Best_Weights, self).__init__()
      self.best = np.Inf
      self.metric = metric
      self.min = min
      if not min:
        self.best = self.best * - 1


  def on_epoch_end(self, epoch, logs=None):
      current_metric = logs.get(self.metric)
      if self.min:
        if np.less(current_metric, self.best):
            self.best = current_metric            
            self.best_weights = model.get_weights() 
            self.name = 'ep_' + str(epoch) + '_' + self.metric + "_" + str(self.best)
            print('saved new ' + self.metric)
      else:
        if np.greater(current_metric, self.best):
            self.best = current_metric            
            self.best_weights = model.get_weights() 
            self.name = 'ep_' + str(epoch) + '_' + self.metric + "_" + str(self.best)
            print('saved new ' + self.metric) 


In [ ]:
batch_size = 32
epochs = 100

if not os.path.exists(root_dir):
    os.makedirs(root_dir)


for idx, view in enumerate(views):
    this_dir = os.path.join(root_dir, str(idx) + '_' + view)
    if not os.path.exists(this_dir):
      os.makedirs(this_dir)
    model = build_model(10, 224)
    print(view)
    loss_cp = Save_Best_Weights(
    )

    acc_cp = Save_Best_Weights(
        metric="val_accuracy",
        min=False
    )

    logger = keras.callbacks.CSVLogger(os.path.join(this_dir, 'logs.csv'))
      
    
    batch_x = x[:, idx, ...]
    batch_x_val = x_val[:, idx, ...]

    model.fit(batch_x, y, batch_size, epochs, callbacks=[loss_cp, acc_cp, logger], validation_data=(batch_x_val, y_val))


    #loss saving
    model.set_weights(loss_cp.best_weights)
    model.save(os.path.join(this_dir, loss_cp.name))

    #acc saving
    model.set_weights(acc_cp.best_weights)
    model.save(os.path.join(this_dir, acc_cp.name))
          

      

print('Finished Training')

bottom
Epoch 1/100
113/113 [==============================] - 20s 83ms/step - loss: 1.7825 - accuracy: 0.4483 - val_loss: 1.1604 - val_accuracy: 0.7400
saved new val_loss
saved new val_accuracy
Epoch 2/100
113/113 [==============================] - 7s 60ms/step - loss: 0.7557 - accuracy: 0.7479 - val_loss: 0.7889 - val_accuracy: 0.7675
saved new val_loss
saved new val_accuracy
Epoch 3/100
113/113 [==============================] - 7s 61ms/step - loss: 0.6134 - accuracy: 0.7916 - val_loss: 0.6421 - val_accuracy: 0.7850
saved new val_loss
saved new val_accuracy
Epoch 4/100
113/113 [==============================] - 7s 60ms/step - loss: 0.5244 - accuracy: 0.8249 - val_loss: 0.5938 - val_accuracy: 0.7925
saved new val_loss
saved new val_accuracy
Epoch 5/100
113/113 [==============================] - 7s 60ms/step - loss: 0.5225 - accuracy: 0.8293 - val_loss: 0.5559 - val_accuracy: 0.8125
saved new val_loss
saved new val_accuracy
Epoch 6/100
113/113 [==============================] - 7s 60ms

In [ ]:
import pandas as pd
     
# dictionary of lists  
this_dict = {'training loss': training_loss_list, 'training acc': training_acc_list, 'val loss': val_loss_list, 'val acc':val_acc_list}  
       
df = pd.DataFrame(this_dict) 
    
# saving the dataframe 
df.to_csv(os.path.join(root_dir, 'logs.csv'))

In [ ]:
y_int = y.argmax(1)

In [ ]:
unique, counts = np.unique(y_int, return_counts=True)
dict(zip(unique, counts))

In [ ]:
unique, counts = np.unique(val_y.argmax(1), return_counts=True)
dict(zip(unique, counts))

In [ ]:
np.savez_compressed('/content/drive/MyDrive/Cache/keras/train_val', x=x, y=y, x_val=val_x, y_val=val_y)

In [ ]:
print(val_y)

In [ ]:
print(inputs.shape)

In [ ]:
cp = torch.load(os.path.join(root_dir, 'last.pt'))
model.load_state_dict(cp['state_dict'])
optimizer.load_state_dict(cp['optimizer'])
last_epoch_done = cp['epoch'] - 1

In [ ]:
#continue training


if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs2.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(last_epoch_done, 101):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # for this_view in range(inputs.shape[1]):
          #   if np.random.randint(0, 4) == 0:
          #     inputs[:, this_view, ...] = 1.0

          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)            


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          open(acc_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')